## Begin the ETL 
First of all, we'll begin with the extraction of dataframe from csv file.
This project require some module we have to use with python:
- python v3.10.x or higher
- pandas module
- dotenv for the environment variable management

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

data_dir = os.getenv('AIRFLOW_HOME') + '/data'

demographic_data = pd.read_csv(f'{data_dir}/Demographic_Data.csv')
geographic_data = pd.read_csv(f'{data_dir}/Geographic_Data.csv')

## Merge the two csv files
We merge the tables to collect one table of the same view 

In [2]:
merged_data = geographic_data.merge(demographic_data,'inner','Location')

merged_data

,Location,Altitude (m),Proximity to Industry (km),Population,Density (people/km²),Urbanization (%),Average Income (USD),Education Level (% with Bachelor's or higher)
0,Los Angeles,89,5.0,10039107,3276,89,60000,32
1,Paris,35,3.0,2140526,21383,98,45000,29
2,Tokyo,40,2.0,13929286,6169,100,55000,37
3,Antananarivo,1276,0.5,1391433,3097,69,1000,10
4,Nairobi,1795,1.0,4397073,6000,61,1500,12
5,Lima,1540,1.5,9674755,3220,81,4500,15


## Creating a DataFrame for Longitude and Latitude
Our current dataset does not include longitude and latitude data, so we need to create a new DataFrame to store this information.

In [3]:
# Create a list of dictionaries for each location
locations = [
    {'Location': 'Antananarivo', 'lon': 47.5256, 'lat': -18.91},
    {'Location': 'Los Angeles', 'lon': -118.242766, 'lat': 34.0536909},
    {'Location': 'Paris', 'lon': 2.320041, 'lat': 48.8588897},
    {'Location': 'Nairobi', 'lon': 36.8172, 'lat': -1.2833},
    {'Location': 'Lima', 'lon': -77.0365256, 'lat': -12.0621065},
    {'Location': 'Tokyo', 'lon': 139.762221, 'lat': 35.6821936}
]

# Directly create a DataFrame from the list of dictionaries
lon_and_lat_df = pd.DataFrame(locations)

lon_and_lat_df

,Location,lon,lat
0,Antananarivo,47.525600,-18.910000
1,Los Angeles,-118.242766,34.053691
2,Paris,2.320041,48.858890
3,Nairobi,36.817200,-1.283300
4,Lima,-77.036526,-12.062107
5,Tokyo,139.762221,35.682194


## Merging Geographic Coordinates with the Existing DataFrame
In this section, we will merge a DataFrame containing geographic coordinates (longitude and latitude) with our existing DataFrame. This process will enhance our dataset by associating each location with its corresponding coordinates

In [4]:
new_merged_data = merged_data.merge(lon_and_lat_df,'inner','Location')
new_merged_data

,Location,Altitude (m),Proximity to Industry (km),Population,Density (people/km²),Urbanization (%),Average Income (USD),Education Level (% with Bachelor's or higher),lon,lat
0,Los Angeles,89,5.0,10039107,3276,89,60000,32,-118.242766,34.053691
1,Paris,35,3.0,2140526,21383,98,45000,29,2.320041,48.858890
2,Tokyo,40,2.0,13929286,6169,100,55000,37,139.762221,35.682194
3,Antananarivo,1276,0.5,1391433,3097,69,1000,10,47.525600,-18.910000
4,Nairobi,1795,1.0,4397073,6000,61,1500,12,36.817200,-1.283300
5,Lima,1540,1.5,9674755,3220,81,4500,15,-77.036526,-12.062107


## Load data from Openweathermap and merge with the existent dataframe
Then, get data from api and merge it

In [6]:
import requests as rq
import os
import json
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

# Variable environment for API key
api_key = os.getenv('API_KEY')

# Base URL for the API
base_url = 'http://api.openweathermap.org/data/2.5/air_pollution'

# function to get the datas in api
def get_api_data(row):
    lon = row['lon']
    lat = row['lat']
    
    response = rq.get(f'{base_url}?lat={lat}&lon={lon}&appid={api_key}')
    data = response.json()

    return data

# Get the aqi data from api
def get_aqi_data(row):
    return get_api_data(row)['list'][0]['main']['aqi']  

# Get polluting gases data
def get_components_data(row):
    return get_api_data(row)['list'][0]['components']  

# Add a new column 'AQI' to the DataFrame by applying the get_aqi_data function
new_merged_data['AQI'] = new_merged_data.apply(get_aqi_data, axis=1)

# Add new columns for pollutants by applying the get_components_data function
components_data = new_merged_data.apply(get_components_data, axis=1)

# Convert the list of dictionaries to a DataFrame
components_df = pd.DataFrame(components_data.tolist())

# Concatenate the components DataFrame with the existing DataFrame
merged_df = pd.concat([new_merged_data, components_df], axis=1)

merged_df


,Location,Altitude (m),Proximity to Industry (km),Population,Density (people/km²),Urbanization (%),Average Income (USD),Education Level (% with Bachelor's or higher),lon,lat,AQI,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,Los Angeles,89,5.0,10039107,3276,89,60000,32,-118.242766,34.053691,3,520.71,3.97,81.57,0.62,15.14,22.76,34.30,2.53
1,Paris,35,3.0,2140526,21383,98,45000,29,2.320041,48.858890,4,236.99,0.65,4.50,144.48,3.04,13.01,18.19,2.69
2,Tokyo,40,2.0,13929286,6169,100,55000,37,139.762221,35.682194,3,433.92,0.75,100.08,8.32,46.73,24.64,31.51,0.72
3,Antananarivo,1276,0.5,1391433,3097,69,1000,10,47.525600,-18.910000,2,417.23,0.19,1.48,75.10,1.73,9.39,19.79,5.45
4,Nairobi,1795,1.0,4397073,6000,61,1500,12,36.817200,-1.283300,3,947.95,0.87,9.43,79.39,3.93,26.23,52.14,16.97
5,Lima,1540,1.5,9674755,3220,81,4500,15,-77.036526,-12.062107,3,774.38,12.07,52.78,3.62,44.82,30.43,47.44,9.12


## Save data output
Finally, save the transformed data into csv file

In [13]:
output_data_dir = f'{data_dir}/output'

merged_df.to_csv(f'{output_data_dir}.csv', index=False)